In [1]:
import numpy as np
import matplotlib.pyplot as plt
import k3d
from k3d import matplotlib_color_maps
import timeit 

In [2]:
def angle(C,n,P):
    """
    Input:
    C = position vector of hypothetical source
    P = position vector of detector
    n = normal vector of detector 
    
    Output:
    t = angle between the normal of the detector and 
    the vector from the detector to the source
    """
    
    r = tuple(map(lambda i, j: i - j, C, P)) # r = vector PC
    magn = np.sqrt(np.dot(n,np.transpose(n))) #magnitude of n
    magr = np.sqrt(np.dot(r,np.transpose(r))) #magnitude of r
    t = np.arccos(np.dot(n,np.transpose(r))/(magn*magr)) #dot product identity
    
    return t 
    
def radify(theta):
    t = (theta/360)*(2*np.pi)
    
    return t 


def Gridsolve(X=50,Y=50,Z=27,P1=(0,0,0),P2=(25,0,27),P3=(50,0,0),nx=30,ny=40,nz=20,n1=(0,1,0),n2=(0,1,0),n3=(0,1,0)):
    
    """
    Input (set to default values):
    C = position vector of hypothetical source
    Pi = position vector of detector i
    ni = normal vector of detector i 
    
    dx, dy, dz = step sizes in each direction
    X,Y,Z = box sizes in each dimension
    
    Output:
    Theta = array of angles for each position for each box (P1,P2,P2)
    G = Array of positions used (x,y,z)
    """
    
    #create axis 
    Nx = np.linspace(0,X,num=nx) 
    Ny = np.linspace(0,Y,num=ny)
    Nz = np.linspace(0,Z,num=nz)
    
    dx = X/nx
    dy = Y/ny
    dz = Z/nz
    
    
    #setup arrays for coordinates
    Theta1 = np.zeros((len(Nx),len(Ny),len(Nz)),dtype='float16')
    Theta2 = np.zeros((len(Nx),len(Ny),len(Nz)),dtype='float16')
    Theta3 = np.zeros((len(Nx),len(Ny),len(Nz)),dtype='float16')
    for i in range(0,len(Nx)):                                           #calculate angles for each box
        for j in range(0,len(Ny)):
            for k in range(0,len(Nz)):
                pos = (i*dx+dx/2,j*dy+dy/2,k*dz+dz/2)
                Theta1[i,j,k] = angle(pos,n1,P1)
                Theta2[i,j,k] = angle(pos,n2,P2)
                Theta3[i,j,k] = angle(pos,n3,P3)
            
    return Theta1, Theta2, Theta3

def AngRange(A,e):
    m = A-e
    M = A+e
    
    return (m,M)

def SourcePos(A,P,r,Dimensions):
    
    X,Y,Z = Dimensions
    Nx,Ny,Nz = np.shape(A)
    Source = np.full_like(A,0)
    
    px,py,pz = P
    
    dx = X/Nx
    dy = Y/Ny
    dz = Z/Nz
    
    for i in range(0,Nx):
        for j in range(0,Ny):
            for k in range(0,Nz):
                x = (i+0.5)*dx
                y = (j+0.5)*dy
                z = (k+0.5)*dz
                
                dist = np.sqrt((x-px)**2+(y-py)**2+(z-pz)**2)
                
                if dist <= r:
                    Source[i,j,k]=2
    
    return Source
                    

def Position(A1, A2, A3,X=33,Y=44,Z=27.5,P1=(16.6,0,15.2),P2=(0,0,6),P3=(31.7,0,5),nx=30,ny=40,nz=20,n1=(0,1,0),n2=(0,1,0),n3=(0,1,0)):
    """
    Ai = (minimum angle,maximum angle)
    """
    m1,M1 = A1
    m2,M2 = A2
    m3,M3 = A3
    
    m1 = radify(m1)
    m2 = radify(m2)
    m3 = radify(m3)
    
    M1 = radify(M1)
    M2 = radify(M2)
    M3 = radify(M3)

    T1,T2,T3 = Gridsolve(X,Y,Z,P1,P2,P3,nx,ny,nz,n1,n2,n3)
    Data = np.full_like(T1,0)
    Nx,Ny,Nz = np.shape(T1)

    for i in range(0,Nx):
        for j in range(0,Ny):
            for k in range(0,Nz):
                if m1<T1[i,j,k]<M1:
                    if m2<T2[i,j,k]<M2:
                        if m3<T3[i,j,k]<M3:
                            Data[i,j,k]=1
    
    voxel = (X/nx)*(Y/ny)*(Z/nz)
    return Data,voxel

def Volume(Data,voxel):
    NV = np.sum(Data)
    V = NV * voxel
    x,y,z = np.shape(Data)
    
    T = (x*y*z)*voxel
    
    perc = V/T*100
    
    r = ((3/4)*V/np.pi)**(1/3)
    return V,r,T,perc

In [4]:
"""
Data 23.02.23
"""
#A1 = AngRange(14.72,13.57)
#A2 = AngRange(15.05,12.25)
A3 = AngRange(34.06,5.97)

A1 = (10,32.3)
A2 = (10,30.99)


start = timeit.default_timer()

Data,voxel = Position(A1,A2,A3,X=33,Y=44,Z=27.5,P1=(17.3,0,17.4),P2=(0,0,10.3),P3=(32,0,10.5),nx=90,ny=120,nz=60)

stop = timeit.default_timer()

print('Time taken: ', stop - start)

Time taken:  53.7680795


In [5]:
Dimensions=(33,44,27.5)
Psource=(9,34,2)
radius = 1
Source = SourcePos(Data,Psource,radius,Dimensions)

In [6]:
Data2 = np.swapaxes(Data,0,2)
SourceT = np.swapaxes(Source,0,2)
plt_volume = k3d.volume(Data2,
                            bounds=[0,33,0, 44,0,28],
                            scaling=[1, 1, 1],
                            color_map=matplotlib_color_maps.Turbo,
                            color_range=[0, 2])
plt_volume2 = k3d.volume(SourceT,
                            bounds=[0,33,0, 44,0,28],
                            scaling=[1, 1, 1],
                            color_map=matplotlib_color_maps.Turbo,
                            color_range=[0, 2])
    
source = (9,34,2)



plot = k3d.plot()
plot+= plt_volume
plot+= plt_volume2
plot.display()

Output()

In [7]:
Volume(Data,voxel)

(3659.7570370370363, 9.559926392673376, 39929.99999999999, 9.165432098765432)

In [8]:
print(voxel)

0.06162037037037036


In [9]:
"""
Speed Test
"""
#A1 = AngRange(14.72,13.57)
#A2 = AngRange(15.05,12.25)
A3 = AngRange(34.06,5.97)

A1 = (10,32.3)
A2 = (10,27.3)


start = timeit.default_timer()

speedtest1 = Position(A1,A2,A3,X=33,Y=44,Z=27.5,P1=(17.3,0,17.4),P2=(0,0,10.3),P3=(32,0,10.5),nx=33,ny=44,nz=27)

stop = timeit.default_timer()

print('Time taken: ', stop - start)

Time taken:  3.6670036999999525


In [10]:
print(speedtest1[1])

1.0185185185185186
